# Evolutionary Bidding Agent

## Enviroment

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import random
import math
import copy

import sys
sys.path.append('../')

from Preprocessing import preprocessing
from Preprocessing.single_set import SingleSet

## Data Preprocessing

Features: data.data_features
Targets: data.data_targets (click, bidprice, payprice)

In [2]:
train_data_path = '/Data/train.csv'
train_data = SingleSet(relative_path=train_data_path,use_numerical_labels=True)

val_data_path = '/Data/validation.csv'
val_data = SingleSet(relative_path=val_data_path,use_numerical_labels=True)

test_data_path = '/Data/test.csv'
test_data = SingleSet(relative_path=test_data_path,use_numerical_labels=True)

-- data loaded --
-- data loaded --
-- data loaded --


## Clicks Data

In [3]:
train_data_click = copy.deepcopy(train_data)
val_data_click = copy.deepcopy(val_data)
test_data_click = copy.deepcopy(test_data)


def pandas_to_numpy(data):

    ## features
    features = np.asarray(data.data_features.values)

    ## targets
    if hasattr(data, "data_targets"):
        labels = np.asarray(data.data_targets.values)
        
    return features, labels



## drop unnecessary features
def drop_features(data):
    
    keep_features = ["hour", "useragent", "adexchange", "url", "slotformat", "slotid"]
    for f in data.data_features:
        if f not in keep_features:
            data.data_features.drop(f, axis=1, inplace = True)

drop_features(train_data_click)
drop_features(val_data_click)
drop_features(test_data_click)

x_train_clicks, y_train_clicks = pandas_to_numpy(train_data_click)
x_val_clicks, y_val_clicks = pandas_to_numpy(val_data_click)
x_test_clicks, y_test_clicks = pandas_to_numpy(test_data_click)




def scale_data(x_train_clicks, x_val_clicks, x_test_clicks):
    
    # normalize the data attributes
    from sklearn import preprocessing
    from sklearn.preprocessing import MinMaxScaler

    ## features
    feature_scaler = MinMaxScaler(feature_range=(0, 1))
    feature_scaler.fit(np.concatenate((x_train_clicks, x_val_clicks, x_test_clicks), axis = 0))       

    x_train_clicks = feature_scaler.transform(x_train_clicks)
    x_val_clicks = feature_scaler.transform(x_val_clicks)
    x_test_clicks = feature_scaler.transform(x_test_clicks)
    
    return x_train_clicks, x_val_clicks, x_test_clicks


x_train_clicks, x_val_clicks, x_test_clicks = scale_data(x_train_clicks, x_val_clicks, x_test_clicks)


input_shape_clicks = x_train_clicks.shape[1]
print("input_shape_clicks", input_shape_clicks)

input_shape_clicks 6


## Payprice Data

In [4]:
train_data_payprice = copy.deepcopy(train_data)
val_data_payprice = copy.deepcopy(val_data)
test_data_payprice = copy.deepcopy(test_data)


def pandas_to_numpy(data):

    ## features
    features = np.asarray(data.data_features.values)

    ## targets
    if hasattr(data, "data_targets"):
        labels = np.asarray(data.data_targets.values)
        
    return features, labels


## drop unnecessary features
def drop_features(data):
    
    keep_features = ["adexchange", "domain", "slotwidth", "slotheight", "slotformat", "slotprice"]
    for f in data.data_features:
        if f not in keep_features:
            data.data_features.drop(f, axis=1, inplace = True)
    

drop_features(train_data_payprice)
drop_features(val_data_payprice)
drop_features(test_data_payprice)

x_train_payprice, y_train_payprice = pandas_to_numpy(train_data_payprice)
x_val_payprice, y_val_payprice = pandas_to_numpy(val_data_payprice)
x_test_payprice, y_test_payprice = pandas_to_numpy(test_data_payprice)


def scale_data(x_train_payprice, x_val_payprice, x_test_payprice):
    
    # normalize the data attributes
    from sklearn import preprocessing
    from sklearn.preprocessing import MinMaxScaler

    ## features
    feature_scaler = MinMaxScaler(feature_range=(0, 1))
    feature_scaler.fit(np.concatenate((x_train_payprice, x_val_payprice, x_test_payprice), axis = 0))       

    x_train = feature_scaler.transform(x_train_payprice)
    x_val = feature_scaler.transform(x_val_payprice)
    x_test = feature_scaler.transform(x_test_payprice)
    
    return x_train_payprice, x_val_payprice, x_test_payprice


x_train_payprice, x_val_payprice, x_test_payprice = scale_data(x_train_payprice, x_val_payprice, x_test_payprice)

input_shape_payprice = x_train_payprice.shape[1]
print("input_shape_payprice", input_shape_payprice)

input_shape_payprice 6


### Data Target Shapes

In [5]:
# clicks
y_train_clicks = np.reshape(y_train_clicks[:,0], (y_train_clicks.shape[0], 1))  # get first column (clicks)
y_val_clicks = np.reshape(y_val_clicks[:,0], (y_val_clicks.shape[0], 1))  # get first column (clicks)

# payprice
y_train_payprice = np.reshape(y_train_payprice[:,2], (y_train_payprice.shape[0], 1))  # get third column (payprice)
y_val_payprice = np.reshape(y_val_payprice[:,2], (y_val_payprice.shape[0], 1))  # get third column (payprice)

# Create Evolutionary Bidding Agents

In [8]:
class Bidding_Agent():
    
    nextid = 0
    
    def __init__(self,):
        
        ## -- Initialize Bidding Agent --
        self.weights_clicks = np.random.uniform(low=-0.1, high=0.1, size=(input_shape_clicks,))
        self.weights_payprize = np.random.uniform(low=-0.1, high=0.1, size=(input_shape_payprice,))
        self.bias = np.random.uniform(low=-0.1, high=0.1, size=(1,))
        
        self.id = Bidding_Agent.nextid
        Bidding_Agent.nextid += 1
        
        
    def improve(self, new_weights_clicks, new_weights_payprice, new_bias, permutation_degree): 
        penalty_clicks = np.random.uniform(low=1-permutation_degree, high=1+permutation_degree, size=(new_weights_clicks.shape[0],))
        penalty_payprize = np.random.uniform(low=1-permutation_degree, high=1+permutation_degree, size=(new_weights_payprice.shape[0],))
        penalty_bias = np.random.uniform(low=1-permutation_degree, high=1+permutation_degree, size=(new_bias.shape[0],))

        self.weights_clicks = new_weights_clicks * penalty_clicks
        self.weights_payprize = new_weights_payprice * penalty_payprize
        self.bias = new_bias * penalty_bias

        
    def place_bids(self, click_features, payprice_features):    
        bids =  payprice_features.dot(self.weights_payprize) + self.bias # +  (click_features.dot(self.weights_clicks)
        #print("average bidprice:", np.mean(bids))
        bids = bids.astype(int)
        return np.reshape(bids, (bids.shape[0],1))
    
    
    def update_stats(self,total_paid, total_clicks, ctr, left_budget):
        self.total_paid = total_paid
        self.total_clicks = total_clicks
        self.ctr = ctr
        self.left_budget = left_budget
        self.results = [left_budget, total_clicks, ctr]
        
        
    def get_parameters(self,):
        return self.weights_clicks, self.weights_payprize, self.bias
            
    
    
agent_1 = Bidding_Agent()

# Train Agents

## Global Auction

In [18]:
num_agents = 30
num_evolutions = 30
permutation_degree = 3.0
permutation_degree_change = 1.1

num_rows = 1000000
x_clicks = x_val_clicks[0:num_rows]
x_payprice = x_val_payprice[0:num_rows]

y_clicks = y_val_clicks[0:num_rows]
y_payprice = y_val_payprice[0:num_rows]

def reward_function(total_paid, total_clicks, ctr, left_budget):
    return (10000 * total_clicks) + (0.0555 * left_budget) + (10 * ctr)

### Run

In [19]:
winning_agent = Bidding_Agent()

agent_list = [Bidding_Agent() for i in range(num_agents)]
evolutions = np.arange(0, num_evolutions, 1)
print("num_agents", num_agents, "num_evolutions", num_evolutions, "permutation_degree", permutation_degree, "permutation_degree_change", permutation_degree_change)

for i in evolutions:
    print("\nevolution:", i, "permutation_degree", permutation_degree)
    
    agent_ranking = list()

    for agent_num, agent in enumerate(agent_list):
        bids = agent.place_bids(x_clicks, x_payprice)
        total_impressions, total_paid, total_clicks, ctr, left_budget = simulate_global_auction(agent, bids, y_clicks, y_payprice)
        agent.update_stats(total_paid, total_clicks, ctr, left_budget)
        
        reward = reward_function(total_paid, total_clicks, ctr, left_budget)
        #print("agent:", agent.id, "| reward:", reward, "| impre:", total_impressions, "| left budget:", left_budget, "| clicks:", total_clicks, "| ctr:", ctr)

        agent_ranking.append((agent,reward))
        
        
    agent_ranking.sort(key=lambda elem: elem[1])
    agent_ranking.reverse()
    #print(agent_ranking)



    winning_agent = agent_list[agent_list.index(agent_ranking[0][0])]
    best_weights_clicks = winning_agent.weights_clicks
    best_weights_payprize = winning_agent.weights_payprize
    best_bias = winning_agent.bias
    print("winning_agent:", winning_agent.id, ", results:", winning_agent.results)


    
    
    ## set all agents to winning agents and permute
    for agent_num, agent in enumerate(agent_list):
        agent.improve(best_weights_clicks, best_weights_payprize, best_bias, permutation_degree)
        
        
        
    ## change permutation degree
    permutation_degree = permutation_degree / permutation_degree_change
    

num_agents 30 num_evolutions 30 permutation_degree 3.0 permutation_degree_change 1.1

evolution: 0 permutation_degree 3.0
winning_agent: 75 , results: [6052861, 5, 0.00064]

evolution: 1 permutation_degree 2.727272727272727
winning_agent: 76 , results: [3043086, 27, 0.00063]

evolution: 2 permutation_degree 2.4793388429752063
winning_agent: 86 , results: [2439765, 26, 0.00044]

evolution: 3 permutation_degree 2.253944402704733
winning_agent: 83 , results: [-1811513, 51, 0.00056]

evolution: 4 permutation_degree 2.0490403660952117
winning_agent: 71 , results: [-4922345, 66, 0.00056]

evolution: 5 permutation_degree 1.8627639691774651
winning_agent: 84 , results: [-338292, 48, 0.00061]

evolution: 6 permutation_degree 1.6934217901613318
winning_agent: 65 , results: [1422334, 36, 0.00056]

evolution: 7 permutation_degree 1.5394743546921197
winning_agent: 84 , results: [6250000, 0, 0.0]

evolution: 8 permutation_degree 1.3995221406291996
winning_agent: 82 , results: [5469886, 5, 0.00048]



In [11]:
def simulate_global_auction(agent, bids, y_clicks, y_payprice):
    
    won_impressions_index = np.where(np.greater_equal(bids, y_payprice) == True)[0]
    
    pay_bids = bids[won_impressions_index]
    obtained_clicks = y_clicks[won_impressions_index]

    total_impressions = len(won_impressions_index)
    total_paid = sum(pay_bids)
    total_clicks = sum(obtained_clicks)
    
    try:
        ctr = total_clicks / total_impressions
    except:
        total_paid = [0]
        total_clicks = [0]
        ctr = [0.0]
        total_impressions = 0
        
    left_budget = 6250000 - int(total_paid[0])
                
    return total_impressions, int(total_paid[0]), total_clicks[0], round(ctr[0], 5), left_budget
    

## Iterative Auction

In [138]:
data_path = os.path.abspath(os.pardir + '/Data/validation.csv')
df = pd.read_csv(data_path, na_values=['Na', 'null']).fillna(0)
print(df.shape)

(303925, 25)


In [457]:
x_clicks = x_val_clicks
x_payprice = x_val_payprice

bids = winning_agent.place_bids(x_clicks, x_payprice)

In [471]:
def simulate_auction(bids):

    budget = 6250000

    ## Evaluation Stats_____________

    bids_won = 0
    earned_clicks = 0
    ctr = 0                  # bids_won / earned_clicks
    total_paid = 0
    cpc = 0                  # cost per click


    for index, row in df.iterrows():

        if bids[index] > budget: # check if budget is sufficient for bidprice
            bids[index] = budget

        if budget <= 0:
            print("-- break after auction #", index)
            break

            
        # WON BID ______________________________________________

        if bids[index] >= row['payprice']:     

            bids_won += 1                        # won the bid
            total_paid += row['payprice']        # add amount to total_paid   
            budget = budget - row['payprice']    # substract money from budget

            # CLICK = 1 ______________________________________________

            if row['click'] == 1:    # only reduce money from budget if ad has been clicked

                    earned_clicks += 1                   # earn the click

        if index%100000 == 0:
            print("bid#", index, ", budget:", budget, ", payprice:", row['payprice'], ", bids_won:", bids_won, ", earned_clicks:", earned_clicks, "\n")


    print("__________________________________\n")

    if earned_clicks > 0:
        cpc = total_paid / earned_clicks
    if bids_won > 0:
        ctr = earned_clicks / bids_won

    print("left budget:", budget)
    print("bids_won:", bids_won)
    print("earned clicks:", earned_clicks)
    print("CTR:", ctr)
    print("cost per click:", cpc)

        
simulate_auction(bids)

bid# 0 , budget: 6249977 , payprice: 23 , bids_won: 1 , earned_clicks: 0 

-- break after auction # 95295
__________________________________

left budget: 0
bids_won: 87521
earned clicks: 52
CTR: 0.0005941431199369294
cost per click: 120192.30769230769


## CSV Submission

In [ ]:
data_path_test = os.path.abspath(os.pardir + '/Data/test.csv')
df_test = pd.read_csv(data_path_test, na_values=['Na', 'null']).fillna(0)


In [ ]:
bidprice_series = pd.Series(data = bids, name='bidprice')
submission_df = pd.DataFrame({'bidid': df_test['bidid'],'bidprice':bidprice_series})

# Group Token: QQri5ISZz4Kn
submission_df.to_csv('testing_bidding_price.csv', index = False)
